In [ ]:
# file dove viene ricercato il modello ottimale per prevedere se uno studente sarà attivo all' anno successivo o meno

In [ ]:
import funzioni
from funzioni import *

import settings
from settings import *

In [ ]:
#==================================================================================================================================================================
#                                                                      OPERAZIONI SUL DB
#==================================================================================================================================================================

In [ ]:
%run preprocessing

In [ ]:
#==================================================================================================================================================================
#                                                            SISTEMO DATI TEMPORALI
#==================================================================================================================================================================

In [ ]:
sistemaDatiTemporali(df,MESE_RIFERIMENTO)

In [ ]:
#==================================================================================================================================================================
#                                                            CREAZIONE MATRICOLE
#==================================================================================================================================================================

In [ ]:
#   tolgo colonne inutili
df.drop('DATA_SUP',axis=1,inplace=True)
df.drop('DATA_FREQ',axis=1,inplace=True)
df.drop('DATA_PRIMO_APPELLO',axis=1,inplace=True)
df.drop('AA_OFF_ID',axis=1,inplace=True)
df.drop('PESO',axis=1,inplace=True)
df.drop('DATA_CHIUSURA',axis=1,inplace=True)

In [ ]:
matricole = calcolaMatricole(df)

In [ ]:
#lo utilizzero' per prevedere le iscrizioni al secondo anno
matricoleConCodice=[]
for elem in matricole:
    copia = elem.copy(deep=True)
    matricoleConCodice.append(copia)


In [ ]:
# ELIMINO LE COLONNE INUTILI E CREO DB PER VISUALIZZARE GLI STUDENTI
for elem in matricole:
    elem.drop('STU_ID',axis=1,inplace=True)
    elem.drop('ULTIMO_TENTATIVO',axis=1,inplace=True)
    elem.drop('PRIMO_ANNO',axis=1,inplace=True)

#creo dataframe db per la visualizzazione più comoda
db = []
for elem in matricole:
    temp = elem.copy(deep=True)
    db.append(temp)

for elem in matricole:
    elem.drop('DES',axis=1,inplace=True)

In [ ]:
# VISUALIZZO STRUTTURA DATI
es = 0
print('matricole:')
display(matricole[es])    #   stato attuale
print('db:')
display(db[es])
# print('cfu: ')
# print(cfu[es])

In [ ]:
#esporto matricole
if ESPORTA_DATI:
    esportaMatricole(matricole)

In [ ]:
#esporto cfu
if ESPORTA_DATI:
    esportaCFU(cfu)

In [ ]:
#==================================================================================================================================================================
#                                                                           CALCOLO X,Y
#==================================================================================================================================================================

In [ ]:
# calcolo X

X = matricole.copy()
print('elementi X: ', len(X))

#   CONVERSIONE A NUMPY ARRAY X
#conversione X
temp = [np.array(X[0].to_numpy().flatten())]
for i in range(1,len(X)):
    temp = np.append(temp,[X[i].to_numpy().flatten()],axis=0)

X = temp

print('tutto X:')
print(X)
print('-------------')
print('singola riga X:')
print(X[0])

In [ ]:
# creazione y

#   creo array che mi dice per ciascuno studente se e' iscritto al secondo anno o meno.
#   se lo studente ha provato almeno un esame lo considero iscritto, altrimenti no.

In [ ]:
#CARICO I DATI

dfOriginale = pd.read_excel('data/Informatica.xlsx')
dfOriginale = dfOriginale[colonneUtilizzate]
dfOriginale

In [ ]:
matricoleConCodice[0]

In [ ]:
#vado a guardare classificare gli studenti tra attivi al II anno (se provano almeno un esame in tutto l' anno), oppure in inattivi

attiviSecondoAnno = []

#indica il numero di studenti iscritti all' anno successivo nel 2016,17,18
# (lo uso solo per fare analisi sui dati)
attiviNegliAnni = [0,0,0]

for i in range(len(matricoleConCodice)):

    #trovo id studente
    idStudente = matricoleConCodice[i].iloc[0,0]
    primoAnno = matricoleConCodice[i].iloc[0,4]

    attivoSecondoAnno = False
    #cerco un esame con l' id dello studente che lo studente ha provato o superato al secondo anno
    for j in range(len(dfOriginale)):
        if dfOriginale.loc[j,'STU_ID'] == idStudente:

            annoMaxAppello = dfOriginale.loc[j,'DATA_MAX_APPELLO']
            annoMaxAppello = str(annoMaxAppello)[0:4]
            
            annoSup = dfOriginale.loc[j,'DATA_SUP']
            annoSup = str(annoSup)[0:4]
            
            annoPrimoApp = dfOriginale.loc[j,'DATA_PRIMO_APPELLO']
            annoPrimoApp = str(annoPrimoApp)[0:4]
            
            if annoMaxAppello == str(primoAnno+1) or annoSup == str(primoAnno+1) or annoPrimoApp == str(primoAnno+1):
                attivoSecondoAnno=True
    
    if attivoSecondoAnno:
        attiviSecondoAnno.append(1)
        if primoAnno == 2016: attiviNegliAnni[0]+=1
        elif primoAnno == 2017: attiviNegliAnni[1]+=1
        elif primoAnno == 2018: attiviNegliAnni[2]+=1
    else:
        attiviSecondoAnno.append(0)
    
print('studenti attivi nell\' anno successivo nel:')
print('2016: ',attiviNegliAnni[0])
print('2017: ',attiviNegliAnni[1])
print('2018: ',attiviNegliAnni[2])


In [ ]:
# calcolo il nuovo y
y = attiviSecondoAnno.copy()

In [ ]:
#conversione y
y = np.array(y)

In [ ]:
#===============bilanciamento dataset

In [ ]:
somma1 = 0
somma0 = 0
for elem in y:
    if elem == 1: somma1+=1
    else: somma0 +=1

perc0 = round(somma0/len(y)*100,2)
perc1 = round((100 - perc0),2)
print('studenti non iscritti II anno',somma0)
print('studenti iscritti II anno: ',somma1)

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = 'Iscritti II anno\n'+str(somma0)+'('+str(perc0)+'%)', 'Non iscritti II anno\n'+str(somma1)+'('+str(perc1)+'%)'
sizes = [somma0, somma1]

fig1, ax1 = plt.subplots()
# ax1.pie(sizes, labels=labels,startangle=90,colors=[ROSSO,VERDE],wedgeprops={"edgecolor":"k",'linewidth': 2,'antialiased': True},textprops={'fontsize': 20})
ax1.pie(sizes,startangle=90,colors=[ROSSO,VERDE],wedgeprops={"edgecolor":"k",'linewidth': 2,'antialiased': True},textprops={'fontsize': 20})
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.savefig('generatedImages/sbilanciamentoDataset2.png', dpi=300,bbox_inches='tight',facecolor='w')
plt.show()


In [ ]:
# oversampling

from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X,y)
X = X_sm
y = y_sm

In [ ]:
somma1 = 0
somma0 = 0
for elem in y:
    if elem == 1: somma1+=1
    else: somma0 +=1

perc0 = round(somma0/len(y)*100,2)
perc1 = round((100 - perc0),2)
print('studenti inattivi II anno',somma0)
print('studenti attivi II anno: ',somma1)

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = 'Iscritti II anno\n'+str(somma0)+'('+str(perc0)+'%)', 'Non iscritti II anno\n'+str(somma1)+'('+str(perc1)+'%)'
sizes = [somma0, somma1]

fig1, ax1 = plt.subplots()
# ax1.pie(sizes, labels=labels,startangle=90,colors=[ROSSO,VERDE],wedgeprops={"edgecolor":"k",'linewidth': 2,'antialiased': True},textprops={'fontsize': 20})
ax1.pie(sizes, startangle=90,colors=[ROSSO,VERDE],wedgeprops={"edgecolor":"k",'linewidth': 2,'antialiased': True},textprops={'fontsize': 20})
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.savefig('generatedImages/bilanciamentoDataset2.png', dpi=300,bbox_inches='tight',facecolor='w')
plt.show()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.75, test_size=0.25,random_state=5)

#   DIMENSIONI X, y
print('dimensioni X: ',X.shape)
print('dimensioni y: ',y.shape)
print('\ndimensioni X_train: ',X_train.shape)
print('dimensioni y_train: ',y_train.shape)
print('dimensioni X_test: ',X_test.shape)
print('dimensioni y_test: ',y_test.shape)

In [ ]:
#calcolo percentuale di studenti che super i 40 cfu in train set
counts = [0,0]
for i in range(len(y_train)):
    if y_train[i]==0:counts[0]+=1
    else:counts[1]+=1

print(' percentuale di studdenti iscritti al II anno in train set: ',counts[1]/(counts[0]+counts[1])*100)

#calcolo percentuale di studenti che super i 40 cfu in test set
counts = [0,0]
for i in range(len(y_test)):
    if y_test[i]==0:counts[0]+=1
    else:counts[1]+=1

print(' percentuale di studdenti iscritti al II anno in test set',counts[1]/(counts[0]+counts[1])*100)


In [ ]:
#==================================================================================================================================================================
#                                                            
# 
#                                                                       ricerca modello migliore


#==================================================================================================================================================================

In [ ]:
labelsRisultatiAccuracy = []
risultatiAccuracy = []
risultatiPrecision = []
risultatiRecall = []
risultatiF1 = []

In [ ]:
#==================================================================================================================================================================
#                                                                     RETE NEURALE
#==================================================================================================================================================================

In [ ]:
#normalizzazione input
Xnorm = tf.keras.utils.normalize(X, axis=1)

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(28, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(2, activation=tf.nn.softmax))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#analisi prestazioni
acc, prec, recall, f1 = analizza(0,Xnorm,y,reteNeurale = True,nloop=300)

In [ ]:
#==================================================================================================================================================================
#                                                                     TPOT CLASSIFICAZIONE
#==================================================================================================================================================================

In [ ]:
#   Internally, TPOT uses joblib to fit estimators in parallel. This is the same parallelization framework used by scikit-learn. 
#   But it may crash/freeze with n_jobs > 1 under OSX or Linux as scikit-learn does, especially with large datasets.
#   One solution is to configure Python's multiprocessing module to use the forkserver start method 
#   (instead of the default fork) to manage the process pools. You can enable the forkserver mode globally for your program by putting 
#   the following codes into your main script:

if USE_TPOT:
    import multiprocessing
    if __name__ == '__main__':
        # multiprocessing.set_start_method('forkserver', force=True)

        #   create an instance of TPOT (n_jobs = -1 utilizza tutti i core disponibili)
        # tpot =  TPOTClassifier(generations = 100, population_size = 100, n_jobs = -1)                             #default                                                                      #impostazioni di default
        # tpot =  TPOTClassifier(generations=2, population_size=20, verbosity=2,n_jobs = -1, random_state=50)       #video 

        tpot =  TPOTClassifier(generations = GENERATIONS, population_size = POP_SIZE, n_jobs = -1)  

    #   You can tell TPOT to optimize a pipeline based on a data set with the fit function
    #   The fit function initializes the genetic programming algorithm to find the highest-scoring 
    #   pipeline based on average k-fold cross-validation. Then, the pipeline is trained on the entire
    #   set of provided samples, and the TPOT instance can be used as a fitted model.
        tpot.fit(X_train, y_train)

In [ ]:
if USE_TPOT:
    #   You can then proceed to evaluate the final pipeline on the testing set with the score function:
    # print(tpot.score(X_test, y_test))
    
    
    # analizza(tpot)

    print('il migliore modello che ho trovato è: \n')
    print(tpot.fitted_pipeline_)

    #   Finally, you can tell TPOT to export the corresponding Python code for the optimized pipeline to a text file with the export function:
    #   Once this code finishes running, tpot_exported_pipeline.py will contain the Python code for the optimized pipeline.

    tpot.export('exported_pipeline_classification2.py')

In [ ]:
#==================================================================================================================================================================
#                                             gradient boosting classifier (miglior modello trovato)
#==================================================================================================================================================================

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectFwe, f_classif
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.svm import LinearSVC
from tpot.builtins import StackingEstimator

model = make_pipeline(
    SelectFwe(score_func=f_classif, alpha=0.023),
    StackingEstimator(estimator=LinearSVC(C=15.0, dual=False, loss="squared_hinge", penalty="l1", tol=0.01)),
    GradientBoostingClassifier(learning_rate=0.5, max_depth=8, max_features=0.1, min_samples_leaf=3, min_samples_split=11, n_estimators=100, subsample=0.6500000000000001)
)

In [ ]:
acc, prec, recall, f1 = analizza(model,X,y)
risultatiAccuracy.append(acc)
risultatiPrecision.append(prec)
risultatiRecall.append(recall)
risultatiF1.append(f1)
labelsRisultatiAccuracy.append('gradient boosting classifier')
m=matrix(model,X_train,y_train,X_test,y_test,regression=False,tipoPrevisioneCFU=False)


In [ ]:
# risultati = importanzaFeaturesMedie(10,model,X,y,regression=False)
# risultati[0]

In [ ]:
#==================================================================================================================================================================
#                                             random forest classifier pipeline (trovata da tpot per questo problema)

                                                        
#==================================================================================================================================================================

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from tpot.builtins import ZeroCount

# Average CV score on the training set was: 0.8868686868686868``
model = make_pipeline(
    RobustScaler(),
    ZeroCount(),
    MinMaxScaler(),
    RandomForestClassifier(bootstrap=True, criterion="entropy", max_features=0.3, min_samples_leaf=12, min_samples_split=13, n_estimators=100)
)

In [ ]:
acc, prec, recall, f1 = analizza(model,X,y)
risultatiAccuracy.append(acc)
risultatiPrecision.append(prec)
risultatiRecall.append(recall)
risultatiF1.append(f1)
labelsRisultatiAccuracy.append('random forest classifier pipeline')
m=matrix(model,X_train,y_train,X_test,y_test,regression=False,tipoPrevisioneCFU=False)


In [ ]:
# importanzaFeaturesMedie(2,model,X,y,regression=False)

In [ ]:
# ANALISI IMPORTANZA FEATURES
# importanze, importanzePos, importanzeNeg = importanzaFeaturesMedie(10,modello,X,y,regression=False)

In [ ]:
#==================================================================================================================================================================
#                                                                           RANDOM FOREST
#==================================================================================================================================================================

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

training_features = X_train
testing_features = X_test
training_target = y_train
testing_target = y_test

# Average CV score on the training set was: 0.8806451612903226
model = RandomForestClassifier(bootstrap=False, criterion="gini", max_features=0.3, min_samples_leaf=17, min_samples_split=5, n_estimators=100)
# Fix random state in exported estimator
if hasattr(model, 'random_state'):
    setattr(model, 'random_state', 50)


In [ ]:
acc, prec, recall, f1 = analizza(model,X,y)
risultatiAccuracy.append(acc)
risultatiPrecision.append(prec)
risultatiRecall.append(recall)
risultatiF1.append(f1)
labelsRisultatiAccuracy.append('random forest')
m=matrix(model,X_train,y_train,X_test,y_test,regression=False,tipoPrevisioneCFU=False)


In [ ]:
#==================================================================================================================================================================
#                                                                           DECISION TREE
#==================================================================================================================================================================


In [ ]:
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
#   dato che non fornisco alcuna indicazione sulla massima profondità dell' albero, avremo probabilmente la massima profondita' possibile
#   di conseguenza andremo probabilment incontro ad overfitting

clf = DecisionTreeClassifier(random_state=0)

In [ ]:
#   cos_complexity_pruning = funzione che mi restituisce due valori: alphas ed impurities
path = clf.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

In [ ]:
#   la funzione ci ha restituito questi valori, che corrispondono ai "weak points" rispetto alle foglie
#   questi valori sono un parametro che posso passare al metodo che definisce il decision tree
ccp_alphas

In [ ]:
#   per ogni alpha che ho calcolato, addestro un albero, e lo salvo nell' array di alberi "clfs"
clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha)
    clf.fit(X_train, y_train)
    clfs.append(clf)
print("Number of nodes in the last tree is: {} with ccp_alpha: {}".format(
      clfs[-1].tree_.node_count, ccp_alphas[-1]))

In [ ]:
#       per ogni albero che ho costruito, calcolo il suo score
#       in questo modo riesco a calcolare il valore di alpha ottimale
#       dove gli score sul train set sono molto alti, ho probabilmente overfitting
#       il punto migliore è quello con il massimo risultato sul test set ed un valore piu o meno simile per il train

train_scores = [clf.score(X_train, y_train) for clf in clfs]
test_scores = [clf.score(X_test, y_test) for clf in clfs]

#       visione ampia

fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("accuracy")
ax.set_title("Accuracy vs alpha for training and testing sets")
ax.plot(ccp_alphas, train_scores, marker='o', label="train",
        drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker='o', label="test",
        drawstyle="steps-post")
ax.legend()
plt.show()

In [ ]:
#   zoom
fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("accuracy")
ax.set_title("Accuracy vs alpha for training and testing sets")
ax.plot(ccp_alphas, train_scores, marker='o', label="train",
        drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker='o', label="test",
        drawstyle="steps-post")
plt.xlim([0.005, 0.05])
ax.legend()
plt.show()

In [ ]:
# TEST SPECIFICO SUGLI ALPHA NEL RANGE PIU' INTERESSANTE

#0100
clf = DecisionTreeClassifier(random_state=0, ccp_alpha=0.0100)
clf.fit(X_train,y_train)

pred=clf.predict(X_test)
acc_0100 = accuracy_score(y_test, pred)

#0125
clf = DecisionTreeClassifier(random_state=0, ccp_alpha=0.0125)
clf.fit(X_train,y_train)

pred=clf.predict(X_test)
acc_0125 = accuracy_score(y_test, pred)

#0150
clf = DecisionTreeClassifier(random_state=0, ccp_alpha=0.0150)
clf.fit(X_train,y_train)

pred=clf.predict(X_test)
acc_0150 = accuracy_score(y_test, pred)

#0175
clf = DecisionTreeClassifier(random_state=0, ccp_alpha=0.0175)
clf.fit(X_train,y_train)

pred=clf.predict(X_test)
acc_0175 = accuracy_score(y_test, pred)

#020
clf = DecisionTreeClassifier(random_state=0, ccp_alpha=0.020)
clf.fit(X_train,y_train)

pred=clf.predict(X_test)
acc_0200 = accuracy_score(y_test, pred)

print('acc 01: ',acc_0100)
print('acc 0125: ',acc_0125)
print('acc 0150: ',acc_0150)
print('acc 0175: ',acc_0175)
print('acc 02: ',acc_0200)


In [ ]:
#   ADDESTRAMENTO ALBERO PRUNED
# model = DecisionTreeClassifier(random_state=0, ccp_alpha=0.02)
model = DecisionTreeClassifier(random_state=0, ccp_alpha=0.01)

In [ ]:
acc, prec, recall, f1 = analizza(model,X,y)
risultatiAccuracy.append(acc)
risultatiPrecision.append(prec)
risultatiRecall.append(recall)
risultatiF1.append(f1)
labelsRisultatiAccuracy.append('decision tree')
m=matrix(model,X_train,y_train,X_test,y_test,regression=False,tipoPrevisioneCFU=False)

In [ ]:
#   stampo albero potato

from sklearn import tree
plt.figure(figsize=(15,10))
tree.plot_tree(clf,filled=True)

In [ ]:
#==================================================================================================================================================================
#                                                                           SVC
#==================================================================================================================================================================

In [ ]:
#addestramento
from sklearn.svm import SVC
model = SVC(gamma='auto', probability=True)

In [ ]:
acc, prec, recall, f1 = analizza(model,X,y)
risultatiAccuracy.append(acc)
risultatiPrecision.append(prec)
risultatiRecall.append(recall)
risultatiF1.append(f1)
labelsRisultatiAccuracy.append('SVC')
m=matrix(model,X_train,y_train,X_test,y_test,regression=False,tipoPrevisioneCFU=False)


In [ ]:
#==================================================================================================================================================================
#                                                                           NAIVE BAYES
#==================================================================================================================================================================

In [ ]:
#   DEFINIZIONE MODELLO
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [ ]:
acc, prec, recall, f1 = analizza(model,X,y)
risultatiAccuracy.append(acc)
risultatiPrecision.append(prec)
risultatiRecall.append(recall)
risultatiF1.append(f1)
labelsRisultatiAccuracy.append('naive bayes')
m=matrix(model,X_train,y_train,X_test,y_test,regression=False,tipoPrevisioneCFU=False)

In [ ]:
#==================================================================================================================================================================
#                                                                     NEAREST NEIGHBOUR
#==================================================================================================================================================================

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=8)

In [ ]:
acc, prec, recall, f1 = analizza(model,X,y)
risultatiAccuracy.append(acc)
risultatiPrecision.append(prec)
risultatiRecall.append(recall)
risultatiF1.append(f1)
labelsRisultatiAccuracy.append('nearest neighbour')
m=matrix(model,X_train,y_train,X_test,y_test,regression=False,tipoPrevisioneCFU=False)


In [ ]:
#==================================================================================================================================================================
#                                                                  SGD
#==================================================================================================================================================================

In [ ]:
from sklearn.linear_model import SGDClassifier
model = SGDClassifier(loss="hinge", penalty="l2", max_iter=100)

In [ ]:
acc, prec, recall, f1 = analizza(model,X,y)
risultatiAccuracy.append(acc)
risultatiPrecision.append(prec)
risultatiRecall.append(recall)
risultatiF1.append(f1)
labelsRisultatiAccuracy.append('SGD')
m=matrix(model,X_train,y_train,X_test,y_test,regression=False,tipoPrevisioneCFU=False)


In [ ]:
#==================================================================================================================================================================
#                                                                    
# 
# 
#                                                       REGRESSIONE
#
#
#
#==================================================================================================================================================================

In [ ]:
#==================================================================================================================================================================
#                                                                           TPOT REGRESSIONE
#==================================================================================================================================================================

In [ ]:
#   Internally, TPOT uses joblib to fit estimators in parallel. This is the same parallelization framework used by scikit-learn. 
#   But it may crash/freeze with n_jobs > 1 under OSX or Linux as scikit-learn does, especially with large datasets.
#   One solution is to configure Python's multiprocessing module to use the forkserver start method 
#   (instead of the default fork) to manage the process pools. You can enable the forkserver mode globally for your program by putting 
#   the following codes into your main script:

if USE_TPOT:
    from tpot import TPOTRegressor
    import multiprocessing
    if __name__ == '__main__':
        multiprocessing.set_start_method('forkserver', force=True)

        #   create an instance of TPOT (n_jobs = -1 utilizza tutti i core disponibili)
        # tpot =  TPOTClassifier(generations = 100, population_size = 100, n_jobs = -1)                             #default                                                                      #impostazioni di default
        # tpot =  TPOTClassifier(generations=2, population_size=20, verbosity=2,n_jobs = -1, random_state=50)       #video 

        tpot =  TPOTRegressor(generations = 80, population_size = 80, n_jobs = -1)  

    #   You can tell TPOT to optimize a pipeline based on a data set with the fit function
    #   The fit function initializes the genetic programming algorithm to find the highest-scoring 
    #   pipeline based on average k-fold cross-validation. Then, the pipeline is trained on the entire
    #   set of provided samples, and the TPOT instance can be used as a fitted model.
        tpot.fit(X_train, y_train)

In [ ]:
if USE_TPOT:
    print('il migliore modello che ho trovato è: \n')
    print(tpot.fitted_pipeline_)

    #   Finally, you can tell TPOT to export the corresponding Python code for the optimized pipeline to a text file with the export function:
    #   Once this code finishes running, tpot_exported_pipeline.py will contain the Python code for the optimized pipeline.

    tpot.export('exported_pipeline_regression2.py')

In [ ]:
#==================================================================================================================================================================
#                                                            REGRESSIONE LINEARE
#==================================================================================================================================================================

In [ ]:
from sklearn import linear_model
model = linear_model.LinearRegression()

In [ ]:
acc, prec, recall, f1 = analizzaRegr(model,X,y)
risultatiAccuracy.append(acc)
risultatiPrecision.append(prec)
risultatiRecall.append(recall)
risultatiF1.append(f1)
labelsRisultatiAccuracy.append('linear regression')
m=matrix(model,X_train,y_train,X_test,y_test,regression=True,tipoPrevisioneCFU=False)


In [ ]:
#==================================================================================================================================================================
#                                                            REGRESSIONE LOGISTICA
#==================================================================================================================================================================

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [ ]:
acc, prec, recall, f1 = analizzaRegr(model,X,y)
risultatiAccuracy.append(acc)
risultatiPrecision.append(prec)
risultatiRecall.append(recall)
risultatiF1.append(f1)
labelsRisultatiAccuracy.append('logistic regression')
m=matrix(model,X_train,y_train,X_test,y_test,regression=True,tipoPrevisioneCFU=False)


In [ ]:
#==================================================================================================================================================================
#                                                            Extra tree regressor
#==================================================================================================================================================================

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import RobustScaler
from tpot.builtins import StackingEstimator, ZeroCount
from sklearn.preprocessing import FunctionTransformer
from copy import copy

# Average CV score on the training set was: -0.10277989254463067
model = make_pipeline(
    make_union(
        RobustScaler(),
        FunctionTransformer(copy)
    ),
    StackingEstimator(estimator=ExtraTreesRegressor(bootstrap=False, max_features=0.9000000000000001, min_samples_leaf=17, min_samples_split=4, n_estimators=100)),
    ZeroCount(),
    StackingEstimator(estimator=RandomForestRegressor(bootstrap=True, max_features=0.05, min_samples_leaf=2, min_samples_split=19, n_estimators=100)),
    ExtraTreesRegressor(bootstrap=False, max_features=0.7500000000000001, min_samples_leaf=9, min_samples_split=19, n_estimators=100)
)

In [ ]:
acc, prec, recall, f1 = analizzaRegr(model,X,y)
risultatiAccuracy.append(acc)
risultatiPrecision.append(prec)
risultatiRecall.append(recall)
risultatiF1.append(f1)
labelsRisultatiAccuracy.append('extra tree regressor')
m=matrix(model,X_train,y_train,X_test,y_test,regression=True,tipoPrevisioneCFU=False)


In [ ]:
#=====================================================================================

#=====================================================================================

In [ ]:
print(labelsRisultatiAccuracy)

In [ ]:
# #scrivo i risultati sul file "performance 1"
# with open('performance2.txt', 'a') as f:
#     f.write("%s\n" % MESE_RIFERIMENTO)
#     for item in risultatiAccuracy:
#         f.write("%s " % item)
#     f.write('\n')
#     for item in risultatiPrecision:
#         f.write("%s " % item)
#     f.write('\n')
#     for item in risultatiRecall:
#         f.write("%s " % item)
#     f.write('\n')
#     for item in risultatiF1:
#         f.write("%s " % item)
#     f.write('\n')